## Pattern Recognition - First Project Report

#### The Task
The task at hand is a supervised prediction task.
We were given some data of a phone, sorted by time. The data included the Batery status and change between two time steps, as well as other features. Given those features we want to predict the change in batterylevel in a few timesteps. This shall be done by using the tools given by the lecture.

#### Preprocessing the data

This step is mainly concerned with creating representations of the data which are suitable for use in the two system components. For the prediction, we need to assign each *unique* state an integer label to index the required transition counts and transition probability matrices. This should be done without any redundance, as the dimensionality of these matrices determines memory and computing costs. As such, a base 2 representation is not suitable. 

For regression, however, we require the original binary activity vectors, as they are multiplied directly by the weights fitted by the linear regression. However, we cannot identify these activity vectors just by their unique state label, as even otherwise equivalent states can have different battery level deltas at the time they were observed. 

Overall, each state therefore recieves one (not necessarily unique) integer label to interface with the transition prediction process, but its original index is also preserved for use in the regression component. Finally, for cross-validation, each data point is linked to the day it was recorded. This allows partioning of the dataset by days, assuming user activity resets roughly at the end of each day.

In [1]:
from utils import *
np.random.seed(123)
dataset_df = get_table().dropna()
mask = (dataset_df['battery_plugged'] == 0) | (dataset_df['battery_plugged'] == 1)
dataset_df = dataset_df[mask]
# month in 8-12, 1-3, day in 1-31
# the following replacements keep 'monthday' chronologically sorted when hashed later
dataset_df['month'][dataset_df['month'] == 1] = 13
dataset_df['month'][dataset_df['month'] == 2] = 14
dataset_df['month'][dataset_df['month'] == 3] = 15
dataset_df['monthday'] = dataset_df['month']*100 + dataset_df['day']

text = 'packages_running_'
keep = [i for i in dataset_df.columns if text in i] + ['battery_plugged'] + ['battery_level'] + ['slot'] + ['monthday']
dataset_df = dataset_df[keep[:49] + keep[50:54] + keep[55:]]
dataset_df = dataset_df.dropna().T.drop_duplicates().T.reset_index()
dataset_df['md_key'] = hash_states(dataset_df['monthday'].to_numpy()[:,None])
dataset_df = dataset_df.drop(['monthday', 'slot'], axis=1)
dataset_df = dataset_df.drop(['packages_running_android', 'packages_running_com.android.calculator2',\
                             'packages_running_com.android.keychain','packages_running_com.android.packageinstaller',\
                             'packages_running_com.android.providers.applications', 'packages_running_com.android.providers.downloads',\
                             'packages_running_com.google.android.email', 'packages_running_edu.udo.cs.ess.mobidac.target',\
                             'packages_running_org.openintents.filemanager', 'packages_running_stream.android'], axis=1)

# get indices of dataset elements per day, so that we can use this partitioning of the data in training and validation
num_days = dataset_df['md_key'].to_numpy().max() + 1
# by day is a list that for each day, contains all dataset indices for that day
by_day = [np.array(dataset_df.index[dataset_df['md_key'] == i].tolist()) for i in range(num_days)]
# keep only days with at least 5 samples
by_day_filtered = [item for item in by_day if len(item) > 4]
# we can access day i by calling dataset_df.loc[by_day[i]]

# in this state space, battery plugged is the last column: activity_vectors[:,-1]
activity_vectors = dataset_df.drop(['index', 'battery_level', 'md_key'], axis=1).to_numpy()
targets = dataset_df['battery_level'].to_numpy()
print('Activity vectors shape:', activity_vectors.shape)
print('Targets shape:', targets.shape)

Activity vectors shape: (2445, 35)
Targets shape: (2445,)


#### State space transforms

As the State space contains quite many different states and computing the makrov chain depends quadratically on those, we desire a low number of unique states. As this comes at a cost we want to minimise the error induced by a reduction of the states. Therefore we compared different pruning methods and decided on one reduction method.  
We firstly threw out all features which were constant through out all examples, as they don't contain any information. Considering this state space we computed the Information gain ratio and the Gini gain and kept the best $\mathcal{N}$ features, where $\mathcal{N}\in[1,\textsf{#FEATURES}]$. In consequence, larger portions of the state space can collapse to the same representative.

In [2]:
targets_binary_1 = (np.sign(targets)==1).astype(int)[None]

state_space_transform_mode = 'Gini'
states = state_space_transform(activity_vectors, targets=targets_binary_1, mode=state_space_transform_mode)
keep_best = 12

states = np.concatenate([states[:,:keep_best], states[:, -1, None]], axis=1)
out_labels_full = hash_states(activity_vectors)
num_unique_states_full = out_labels_full.max()+1
print('Number of unique states without pruning:', num_unique_states_full)

out_labels_pruned = hash_states(states)
num_unique_states_pruned = out_labels_pruned.max()+1
print('Number of unique states:', num_unique_states_pruned)

Number of unique states without pruning: 637
Number of unique states: 291


#### Prediction:

The input to the prediction system should be sequences of observed transitions between states. These are readily represented by lists of lists containing state labels. Subsequent state labels $i$ and $j$ within the same (lower-level) list represent an oberved transition from state $i$ to state $j$, and a matrix $N =(n)_{j,i}$ counting the state transitions is incremented accordingly. Aggregated over all lists,  $N$ counts all observed transitions in the data set, while also respecting reset of user behaviour.

##### MLE Prediction

For the MLE predictor, we simply normalize $N$ to the column-stochastic matrix $\theta_{MLE}$, which means that the entry 
 $\theta_{MLE}$ at index $(j,i)$ represents the frequency of state transitions to the state with the label $j$ given that the current state has the label $i$. 

##### MAP Prediction

The MAP prediction process is much more involved and requires approximation of the expectation $E[\theta,\mathcal{D}]$ using many randomly generated transition probability matrices $\theta$. These are averaged using a weighting scheme determined by the extent to which they can explain the observed transitions. Overall, individual weights are expected to become extremely small due to the high number of matrix entries. This means that some exp-log tricks with a subsequent shift of the sample weights in logspace are required to achieve runtimes compatible with exhaustive cross-validation. Thus, our code does not look very familiar, but in principle it is based purely on the example Sebastian showed in a previous exercise. 

To justify the usage of MAP prediction given the comparitively high computational costs, regard the following dummy example of a Markov Chain with transition probabilities described by

$\begin{pmatrix}
 \frac{1}{3} & \frac{1}{4} & \frac{1}{5} & \frac{1}{6} \\
 \frac{1}{6} & \frac{1}{4} & \frac{1}{5} & \frac{1}{3} \\
 \frac{1}{3} & \frac{1}{4} & \frac{2}{5} & \frac{1}{6} \\
 \frac{1}{6} & \frac{1}{4} & \frac{1}{5} & \frac{1}{3}
\end{pmatrix}$

for which we simulate a trajectory containing a small number of state transitions. 

In [3]:
# a minimal example of the MLE/MAP prediction process
# first, we define some ground truth transition matrix for a dummy Markov Chain
gt = np.array([[1./3., 1./4., 1./5., 1./6.], [1./6., 1./4., 1./5, 1./3.], [1./3., 1./4., 2./5, 1./6.], [1./6., 1./4., 1./5., 1./3.]])
# start in some state, here it is 0
traj = [int(0)]
# get only a small number of sample transitions within this Markov Chain, add them to the 'trajectory'
nn = 29
#simulate the process
for i in range(nn):
    traj.append(np.random.choice([0, 1, 2, 3], p=gt[:,traj[-1]]).astype(int))
traj = np.array(traj).astype(int)
# here we simply split the sequence traj into traj[:15], traj[15:] to emphasize that the input is a list of sequences
predictor_input = [traj[:15], traj[15:]]
print('Predictor input:')
for sequence_observed in predictor_input:
    print(sequence_observed)
start = time.time()
mle_P = fit_predictor(predictor_input, num_unique_states=traj.max()+1, mode='MLE')
map_P = fit_predictor(predictor_input, num_unique_states=traj.max()+1, mode='MAP')
print('Code block execution took', time.time()-start, 'seconds.')
print('Ground Truth:\n', gt, '\n')
print('MLE:\n', mle_P, '\n')
print('MAP:\n', map_P, '\n')

Predictor input:
[0 2 1 0 2 2 2 3 3 1 1 1 2 2 0]
[1 2 0 0 2 2 2 3 3 2 2 1 1 0 0]
Code block execution took 0.0014426708221435547 seconds.
Ground Truth:
 [[0.33333333 0.25       0.2        0.16666667]
 [0.16666667 0.25       0.2        0.33333333]
 [0.33333333 0.25       0.4        0.16666667]
 [0.16666667 0.25       0.2        0.33333333]] 

MLE:
 [[0.4        0.28571429 0.16666667 0.        ]
 [0.         0.42857143 0.16666667 0.25      ]
 [0.6        0.28571429 0.5        0.25      ]
 [0.         0.         0.16666667 0.5       ]] 

MAP:
 [[0.34736398 0.23204555 0.21750678 0.15624641]
 [0.11083901 0.32839146 0.2199273  0.27525666]
 [0.41380775 0.31793639 0.3834216  0.23226108]
 [0.1279891  0.12162656 0.17914411 0.33623573]] 



Conclusion: When the space of state transitions is undersampled, the MLE result is good at reproducing the observed data, but might be very bad at generalizing to predicting unseen state transitions. For a relatively low amount of samples, we expect MAP to achieve better prediction results.

#### Prediction results

Keeping this in mind, we move to apply these methods on the real dataset. First, we use the previous partition of the data into days to perform leave-one-(day)-out cross-validation. Regard the results of the MLE predictor averaged over all validation splits.

<div>
<img src="img/MLE_predictor_bitflip.png" width="400">
</div>

It is evident that the performance, measured in percentage bit-flips between the ground truth state and the predicted state, differs vastly between training and validation sets. While it is acceptable on the training set, the validation set loss clearly shows signs of model overfitting, which is what we expect from MLE prediction since the observed number of state transitions is low.

<div>
<img src="img/MAP_prediction_bitflip.png" width="400">
</div>

MAP prediction, however, generalizes very well from the training to the validation sets on average. Visualizing these errors based on the validation split demonstrates that they are generally much better than a random prediction over all of the train-valid data partitions.

<div>
<img src="img/MAP_prediction_bitflip_by_split.png" heigth="1200">
</div>

Note that we can also measure these errors in terms of the BCE, which shows similar, slightly smoother, albeit more difficult to intepret results. Overall, both terms are mostly interchangeable.

<div>
<img src="img/MAP_prediction_bce_by_split.png" width="1200">
</div>

Therefore, we constrain the evaluation of the state transition prediction to the bit-flip losses.

#### Prediction Output Transforms

Above the prediction output is compared to the activity vectors corresponding to the ground truth state. This is not possible without further consideration, because the output of the prediction is just a probability distribution over the set of unique states. Bluntly put, the activity vector has $\textsf{#FEATURES}$ entries, whereas the prediction output has $\textsf{#UNIQUE STATES}$ entries.

Naively, the most straightforward option would be to take the argmax of the state distribution and then look up the corresponding activity vector. However, this discards alot of useful information, especially in the face of very uncertain state transitions. Therefore, we also experimented with converting the output distribution of the predictor to a set of individual state-space feature activation distributions via a weighted sum over all unique state vectors. The result can also be rounded in each component, yielding a nearest-neighbor fit that can even produce unobserved states.

#### Regression

We implement the regression components in the closed form solutions given in the lecture, which is possible because we took care to remove redundancies in the state space in the preprocessing, guaranteeing invertible hat matrices. Explicitly, the formulas used were as follows:

##### MLE

$w_{\theta_{MLE}} = (X X^T)^{-1} X y$

##### MAP

$w_{\theta_{MAP}} = (X X^T + \sigma Id)^{-1} X y$

with $X$ the data matrix constructed by concatentating the state vectors with a bias vector, the targets $y$ and $\sigma = \operatorname{var}(y)$. The regressor output is then given by

$\hat{y} = X^T w$

#### Regression results

At first glance, the results of the MLE regressor, measured as the absolute error between ground truth battery delta and the regressor output, seems quite acceptable.

<div>
<img src="img/MLE_regression_overview.png" width="400">
</div>

The overall result of the MAP regression does not point out any obvious advantages here.

<div>
<img src="img/MAP_regression_overview.png" width="400">
</div>

However, splitting the sample errors by battery plugged state reveals overfitting in the situations where the phone is plugged in for charging, as the number of samples is not large enough for these states. This is 'hidden' in the overall error, also because the number of samples is small.

<div>
<img src="img/MLE_regression_by_battery.png" width="800">
</div>


As expected, MAP does better for these cases, and generalizes well. When comparing these two plots, note the scale of the y-axis.

<div>
<img src="img/MAP_regression_by_battery.png" width="800">
</div>

#### Target Space Transforms

As the regression only yields a linear function, adjusting the target space to be more suitable for a linear fit might be beneficial. To test this, we mapped the negative battery deltas to the square root of their absolute values. After the regressor is fit, its estimates can be converted by applying the inverse function to $\hat{y}$.

#### Selecting the overall system

At this point, we have many different potential configurations of regressors and predictors, starting with the basic subdivision into MLE or MAP approaches, but also encompassing more subtle choices like state or target space transform modes, how much cropping is done based on the Gini or Informations gains. Evaluation of every combination of hyperparameters is not straightforward, because each trial requires full calculation of the cross-validation scheme. This could amount to, for example, around 110 calculations of the MAP prediction matrix per trial, with hundreds of trials necessary to cover the whole hyperparameter space.

Using a functionality provided by [weights and biases](https://wandb.ai/), we complete this grid search in a distributed manner. Based on the (somewhat shady) assumption that predictor and regressor are independent subsystems, we evaluate all variations of the config shown below.

In [4]:
hyperparameter_defaults = dict(
    state_transform_mode = 'Id', #'Gain', 'Gini'
    keep_best = 16, #1-34
    predictor_mode = 'MLE', #'MAP'
    prediction_transform = 'argmax', #'activity_dist', 'argmax', 'nearest_neighbor'
    regressor_mode = 'MLE', #'MAP'
    charge_transform_mode = 'Id',
    discharge_transform_mode = 'Id', #'Sqrt'
)

#### Results, remaining problems

To evaluate the full system, we arbitrarily select one split into training and validation data as an example. This is justified considering that our components previously showed consistant performance over all training and validation splits. Predictor and regressor are now fitted using exactly the same training data. The predictor generates results over $1$ to $4$ timesteps, which are converted to activity distributions and fed directly into the regressor that was previously fitted to the training data. The absolute error of the battery level changes is aggregated over all timesteps. 

First, we can observe that our system performance is bottle-necked by the prediction accuracy.

<div>
<img src="img/Full_system_bitflips_vs_time.png" width="1200">
</div>

The percentage of bit-flips increases with increasing prediction time horizon, converging to values only slightly better than a random prediction. This reflects the large uncertainty inherent to each prediction step. However, the regressor performs relatively well despite this, indicating that the uncertainty might have been successfully concentrated in less relevant components of the state space.

<div>
<img src="img/Full_system_L1_vs_time.png" width="1200">
</div>

It should be noted that the number of training examples is so low for states in which the battery is being charged that the predictor, over time, basically refused to consider that the state might change from a discharging to a charging one. Therefore, we can only derive predictions for time horizons until which we expect the user to not have access to a charger.

However, for these scenarios, we can predict up to $2$ hours into the future, with total absolute errors of around $4$. Compared to the range of these values in the data, $[-10, 30]$ per interval, this total over up to $4$ intervals seems acceptable. Using both components in the MAP configuration, the system generalizes very well from the training to the validation set.